# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at ntu-ml-2023spring-ta@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/15lGUmT8NpLGtoxRllRWCJyQEjhR1Idcei63YHsDckPE/edit#slide=id.g21fff4e9af6_0_13)　Kaggle: [Link](https://www.kaggle.com/competitions/ml2023spring-hw7/host/sandbox-submissions)　Data: [Link](https://drive.google.com/file/d/1YU9KZFhQqW92Lw9nNtuUPg0-8uyxluZ7/view?usp=sharing)




# Prerequisites

## Download Dataset

In [1]:
# download link 1
# !gdown --id '1TjoBdNlGBhP_J9C66MOY7ILIrydm7ZCS' --output hw7_data.zip

# download link 2 (if above link failed)
# !gdown --id '1YU9KZFhQqW92Lw9nNtuUPg0-8uyxluZ7' --output hw7_data.zip

# download link 3 (if above link failed)
# !gdown --id '1k2BfGrvhk8QRnr9Xvb04oPIKDr1uWFpa' --output hw7_data.zip

# !unzip -o hw7_data.zip

# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

Thu May 16 05:20:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              27W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Install packages

Documentation for the toolkit: 
*   https://huggingface.co/transformers/
*   https://huggingface.co/docs/accelerate/index



In [2]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.26.1
!pip install accelerate==0.16.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.39.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.3 requires transformers>=4.33.1, but you have transformers 4.26.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found e

# Kaggle (Fine-tuning)

## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2hrs
  

## Import Packages

In [3]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	torch.manual_seed(seed)
	if torch.cuda.is_available():
			torch.cuda.manual_seed(seed)
			torch.cuda.manual_seed_all(seed)
	np.random.seed(seed)
	random.seed(seed)
	torch.backends.cudnn.benchmark = False
	torch.backends.cudnn.deterministic = True
same_seeds(2)

2024-05-16 05:20:44.129875: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 05:20:44.130003: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 05:20:44.261465: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Load Model and Tokenizer




 

In [4]:
from transformers import (
  AutoTokenizer,
  AutoModelForQuestionAnswering,
)

# model_name = "luhua/chinese_pretrain_mrc_roberta_wwm_ext_large"
model_name = "luhua/chinese_pretrain_mrc_macbert_large"
model = AutoModelForQuestionAnswering.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

config.json:   0%|          | 0.00/669 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

## Read Data

- Training set: 26918 QA pairs
- Dev set: 2863  QA pairs
- Test set: 3524  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [5]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("/kaggle/input/ml2023spring-hw7/hw7_train.json")
dev_questions, dev_paragraphs = read_data("/kaggle/input/ml2023spring-hw7/hw7_dev.json")
test_questions, test_paragraphs = read_data("/kaggle/input/ml2023spring-hw7/hw7_test.json")

## Tokenize Data

In [6]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

## Dataset

In [7]:
doc_stride = 25

class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 100 # 60  change to 100
        self.max_paragraph_len = 350 #  150 change to 350
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = doc_stride

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn
        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
#             mid = (answer_start_token + answer_end_token) // 2
#             paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
#             paragraph_end = paragraph_start + self.max_paragraph_len
            start_min = max(0, answer_end_token - self.max_paragraph_len + 1) # 把答案放在结尾
            start_max = min(answer_start_token, len(tokenized_paragraph) - self.max_paragraph_len)# 把答案放在开头
            start_max = max(start_min, start_max)
            paragraph_start = random.randint(start_min, start_max + 1)
            paragraph_end = paragraph_start + self.max_paragraph_len

            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

## Function for Evaluation

In [8]:
def evaluate(data, output, doc_stride=32, token_type_ids=None, paragraph=None, paragraph_tokenized=None):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        
        token_type_id = data[1][0][k].detach().cpu().numpy()
        #[CLS] + [question] + [SEP] + [paragraph] + [SEP]
        paragraph_start = token_type_id.argmax()
        paragraph_end = len(token_type_id) - 1 - token_type_id[::-1].argmax() - 1


        # start index 应该比 end index 小 / start index 应该比 paragraph_end 大 / end index 应该比 paragraph_start 小
        if start_index > end_index or start_index < paragraph_start or end_index > paragraph_end:
            continue
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
            # 找到tokenized paragraph中对应的位置
            origin_start = start_index + k * doc_stride - paragraph_start
            origin_end = end_index + k * doc_stride - paragraph_start
    
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    answer = answer.replace(' ','')
    
    # 替换"[UNK]"
    answer = answer.replace("[UNK]", "")

#     # Result 中存在[UNK] 说明存在编码问题 需要修正
#     if '[UNK]' in answer:
#         print('发现 [UNK]，这表明有文字无法编码, 使用原始文本')
# #         print("Paragraph:", paragraph)
# #         print("Paragraph:", paragraph_tokenized.tokens)
#         print('--直接解码预测:', answer)
#         #找到原始文本中对应的位置
#         # 检查origin_start的值
#         if origin_start is None:
#             print('origin_start is None')
#         else:
#             # 检查token_to_chars的返回值
#             token_to_chars_result = paragraph_tokenized.token_to_chars(origin_start)
#             if token_to_chars_result is None:
#                 print('token_to_chars returned None')
#                 print(f'origin st:{origin_start}')
#                 print(f'len para token:{len(paragraph_tokenized)}')
#             else:
#                 raw_start =  paragraph_tokenized.token_to_chars(origin_start)[0]
#                 # 检查origin_end的值
#                 if origin_start is None:
#                     print('origin_end is None')
#                 else:
#                     # 检查token_to_chars的返回值
#                     token_to_chars_result = paragraph_tokenized.token_to_chars(origin_end)
#                     if token_to_chars_result is None:
#                         print('token_to_chars returned None')
#                         print(f'origin ed:{origin_end}')
#                         print(f'len para token:{len(paragraph_tokenized)}')
#                     else:
#                         raw_end =  paragraph_tokenized.token_to_chars(origin_end)[1]
# #         raw_start =  paragraph_tokenized.token_to_chars(origin_start)[0]
#                         raw_end = paragraph_tokenized.token_to_chars(origin_end)[1]
#                         answer = paragraph[raw_start:raw_end]
#                         print('--原始文本预测:',answer)
    
    return answer

In [9]:
test_paragraphs_tokenized.token_to_chars(20)[1]
test_questions[0]['paragraph_id']

997

## Training

In [10]:
from accelerate import Accelerator

# hyperparameters
name = "boss2"
num_epoch = 1
validation = True
logging_step = 100
learning_rate = 1e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)
train_batch_size = 8
lr_sche = False

#### TODO: gradient_accumulation (optional)####
# Note: train_batch_size * gradient_accumulation_steps = effective batch size
# If CUDA out of memory, you can make train_batch_size lower and gradient_accumulation_steps upper
# Doc: https://huggingface.co/docs/accelerate/usage_guides/gradient_accumulation
gradient_accumulation_steps = 16

# dataloader
# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)


# Change "fp16_training" to True to support automatic mixed 
# precision training (fp16)	
fp16_training = True
if fp16_training:    
    accelerator = Accelerator(mixed_precision="fp16")
else:
    accelerator = Accelerator()

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/
model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 
  

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [11]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("./runs/"+name)

In [12]:
# import wandb
# from kaggle_secrets import UserSecretsClient

# user_secrets = UserSecretsClient()
# api_key = user_secrets.get_secret("wandb_api")
# wandb.login(key=api_key)

In [13]:
# proj = "HW7_BERT"
# name = "simple"
# config = {
#     "num_epoch":num_epoch,
#     "logging_step":logging_step,
#     "lr_scheduler":lr_sche,
#     "optim":"AdamW",
#     "train_batchsize":train_batch_size,
#     "grad_accu_steps":gradient_accumulation_steps,
#     "effective_batchsize":train_batch_size*gradient_accumulation_steps,
#     "fp16_train":fp16_training,
#     "GPU_type":"P100" 
# }
# wandb.init(
#     project=proj,
#     name=name,
#     config=config
# )
# # define our custom x axis metric
# wandb.define_metric("train/step")
# # set all other train/ metrics to use this step
# wandb.define_metric("train/*", step_metric="train/step")

In [14]:
total_steps = len(train_loader)

In [15]:
model.train()


print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0
    best_train_acc = 0
    if lr_sche:
        from transformers import get_linear_schedule_with_warmup
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=total_steps)
    for data in tqdm(train_loader):	
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])
        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)
        
        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
           
        train_loss += output.loss
        
        accelerator.backward(output.loss)
        
        step += 1
        ##### TODO: Apply linear learning rate decay #####
        optimizer.step()
        optimizer.zero_grad()
        if lr_sche:
#             optimizer.state_dict()['param_groups'][0]['lr'] -= learning_rate/total_steps
                scheduler.step()
        
        writer.add_scalar("LR", optimizer.state_dict()['param_groups'][0]['lr'], step)

        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            if (train_acc / logging_step)>best_train_acc:
                best_train_acc = (train_acc / logging_step)
            
#             # Use wandb to record
#             log_dict = {
#                 "train/step": step % logging_step,  # exponential growth w/ internal W&B step
#                 "train/accuracy": train_acc / logging_step,  # x-axis is train/step
#                 "train/lr": optimizer.state_dict()['param_groups'][0]['lr'],  # x-axis is train/step
#             }
#             wandb.log(log_dict)
#             wandb.log({"Acc/train":train_acc / logging_step}, step=step % logging_step)
#             wandb.log({"lr":optimizer.state_dict()['param_groups'][0]['lr']}, step=step % logging_step)
            train_loss = train_acc = 0
    ##########################################################
#         if step ==300:
#             break
###############################################################
    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output, doc_stride=doc_stride,paragraph=dev_paragraphs[dev_questions[i]["paragraph_id"]],
                                 paragraph_tokenized=dev_paragraphs_tokenized[dev_questions[i]["paragraph_id"]]) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")            
#             # Use wandb to record
#             wandb.log({"valid/accuracy":dev_acc / len(dev_loader)})
        model.train()

    
writer.add_hparams({
    "model_name":model_name,
    "doc_stride":doc_stride,
    "preprocess":"Random",
    "postprocess":True,
    "lr_decay":lr_sche,
    "fp16_train":fp16_training,
    "optim":"AdamW", 
    "train_bs":train_batch_size, 
    "grad_accu_steps":gradient_accumulation_steps, 
    "effective_train_bs":train_batch_size*gradient_accumulation_steps},
    {
        "Acc_train":best_train_acc,
        "Acc_valid":dev_acc / len(dev_loader)
    },
    run_name=name
)

    
# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "saved_model" 
model.save_pretrained(model_save_dir)

Start Training ...


  0%|          | 0/3365 [00:00<?, ?it/s]

Epoch 1 | Step 100 | loss = 0.992, acc = 0.639
Epoch 1 | Step 200 | loss = 0.706, acc = 0.729
Epoch 1 | Step 300 | loss = 0.702, acc = 0.751
Epoch 1 | Step 400 | loss = 0.667, acc = 0.755
Epoch 1 | Step 500 | loss = 0.714, acc = 0.746
Epoch 1 | Step 600 | loss = 0.655, acc = 0.754
Epoch 1 | Step 700 | loss = 0.719, acc = 0.744
Epoch 1 | Step 800 | loss = 0.758, acc = 0.715
Epoch 1 | Step 900 | loss = 0.666, acc = 0.734
Epoch 1 | Step 1000 | loss = 0.625, acc = 0.759
Epoch 1 | Step 1100 | loss = 0.569, acc = 0.770
Epoch 1 | Step 1200 | loss = 0.660, acc = 0.767
Epoch 1 | Step 1300 | loss = 0.624, acc = 0.745
Epoch 1 | Step 1400 | loss = 0.623, acc = 0.767
Epoch 1 | Step 1500 | loss = 0.709, acc = 0.746
Epoch 1 | Step 1600 | loss = 0.563, acc = 0.762
Epoch 1 | Step 1700 | loss = 0.645, acc = 0.751
Epoch 1 | Step 1800 | loss = 0.564, acc = 0.771
Epoch 1 | Step 1900 | loss = 0.605, acc = 0.759
Epoch 1 | Step 2000 | loss = 0.694, acc = 0.743
Epoch 1 | Step 2100 | loss = 0.626, acc = 0.761
E

  0%|          | 0/2863 [00:00<?, ?it/s]

Validation | Epoch 1 | acc = 0.855
Saving Model ...


## Testing

In [16]:
# test_paragraphs_tokenized[test_questions[286]["paragraph_id"]].token_to_chars(20)[0]

In [17]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for data in tqdm(test_loader):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output,doc_stride=doc_stride, paragraph=test_paragraphs[test_questions[i]["paragraph_id"]],
                               paragraph_tokenized=test_paragraphs_tokenized[test_questions[i]["paragraph_id"]]))

result_file = "result.csv"
with open(result_file, 'w') as f:	
    f.write("ID,Answer\n")
    for i, test_question in enumerate(test_questions):
    # Replace commas in answers with empty strings (since csv is separated by comma)
    # Answers in kaggle are processed in the same way
        f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  0%|          | 0/3524 [00:00<?, ?it/s]

Completed! Result is in result.csv


# GradeScope - Question 2 (In-context learning)

### In-context learning
The example prompt is :
```
請從最後一篇的文章中找出最後一個問題的答案：
文章：<文章1 內容>
問題：<問題1 敘述>
答案：<答案1>
...
文章：<文章n 內容>
問題：<問題n 敘述>
答案：
```

In [18]:
# import torch
# import random  
# import numpy as np

# # To avoid CUDA_OUT_OF_MEMORY
# torch.set_default_tensor_type(torch.cuda.FloatTensor)

# # Fix random seed for reproducibility
# def same_seeds(seed):
# 	torch.manual_seed(seed)
# 	if torch.cuda.is_available():
# 			torch.cuda.manual_seed(seed)
# 			torch.cuda.manual_seed_all(seed)
# 	np.random.seed(seed)
# 	random.seed(seed)
# 	torch.backends.cudnn.benchmark = False
# 	torch.backends.cudnn.deterministic = True
# same_seeds(2)

In [19]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# # You can try model with different size
# # When using Colab or Kaggle, models with more than 2 billions parameters may 
# # run out of memory
# tokenizer = AutoTokenizer.from_pretrained("facebook/xglm-1.7B")
# model = AutoModelForCausalLM.from_pretrained("facebook/xglm-1.7B")

In [20]:
# # To clean model output. If you try different prompts, you may have to fix 
# # this function on your own
# def clean_text(text):
#     # Note: When you use unilingual model, the colon may become fullwidth
#     text = text.split("答案:")[-1]
#     text = text.split(" ")[0]
#     return text

In [21]:
# import random
# import json

# with open("/kaggle/input/ml2023spring-hw7/hw7_in-context-learning-examples.json", "r") as f: 
#     test = json.load(f)

# # K-shot learning 
# # Give model K examples to make it achieve better accuracy 
# # Note: (1) When K >= 4, CUDA_OUT_OFF_MEMORY may occur.
# #       (2) The maximum input length of XGLM is 2048
# K = 2

# question_ids = [qa["id"] for qa in test["questions"]]

# with open("in-context-learning-result.txt", "w") as f:
#     print("ID,Ground-Truth,Prediction", file = f)
#     with torch.no_grad():
#         for idx, qa in enumerate(test["questions"]):
#             # You can try different prompts
#             prompt = "請從最後一篇的文章中找出最後一個問題的答案\n"
#             exist_question_indexs = [question_ids.index(qa["id"])]

#             # K-shot learning: give the model K examples with answers
#             for i in range(K):
#                 question_index = question_ids.index(qa["id"])
#                 while(question_index in exist_question_indexs): 
#                     question_index = random.randint(0, len(question_ids) - 1)
#                 exist_question_indexs.append(question_index)    
#                 paragraph_id = test["questions"][question_index]["paragraph_id"]
#                 prompt += f'文章：{test["paragraphs"][paragraph_id]}\n'
#                 prompt += f'問題：{test["questions"][question_index]["question_text"]}\n'
#                 prompt += f'答案：{test["questions"][question_index]["answer_text"]}\n'

#             # The final one question without answer
#             paragraph_id = qa["paragraph_id"]
#             prompt += f'文章：{test["paragraphs"][paragraph_id]}\n'
#             prompt += f'問題：{qa["question_text"]}\n'
#             prompt += f'答案：'
            
#             inputs = tokenizer(prompt, add_special_tokens=False, return_tensors="pt") 
#             sample = model.generate(**inputs, max_new_tokens = 20)
#             text = tokenizer.decode(sample[0], skip_special_tokens=True)

#             # Note: You can delete this line to see what will happen
#             text = clean_text(text)
            
#             print(prompt)
#             print(f'正確答案: {qa["answer_text"]}')
#             print(f'模型輸出: {text}')
#             print()

#             print(f"{idx},{qa['answer_text']},{text}", file = f)